# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd
import os

try:
    path = os.path.join('datasets/data.csv')
    df = pd.read_csv(path)
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран

In [2]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Давайте посмотрим основную информацию о данных

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` будет заполнен медианным доходом среди всех записей с тем же типом

In [5]:
for t in df['income_type'].unique():
    df.loc[(df['income_type'] == t) & (df['total_income'].isna()), 'total_income'] = \
    df.loc[(df['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными

In [6]:
df['days_employed'] = df['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях

In [7]:
df.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

Выведем перечень уникальных значений столбца `children`

In [8]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения

In [9]:
df = df[(df['children'] != -1) & (df['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены

In [10]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типа занятости `income_type`

In [11]:
for t in df['income_type'].unique():
    df.loc[(df['income_type'] == t) & (df['days_employed'].isna()), 'days_employed'] = \
    df.loc[(df['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены

In [12]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный

In [13]:
df['total_income'] = df['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру

In [14]:
df['education'] = df['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных

In [15]:
df.duplicated().sum()

71

In [16]:
df = df.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

df['total_income_category'] = df['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [18]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'
    
df['purpose_category'] = df['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [20]:
def correlation_pivot(df, index):
    correlation = df.pivot_table(index=[index], values='debt', aggfunc={'count', 'mean'})
    correlation['mean'] = correlation['mean'] * 100
    correlation = correlation.rename(columns={'count': 'общее число заемщиков', 'mean': 'доля должников в %'})
    return correlation

correlation_pivot(df, 'children')

,общее число заемщиков,доля должников в %
children,,
0,14091,7.543822
1,4808,9.234609
2,2052,9.454191
3,330,8.181818
4,41,9.756098
5,9,0.000000


**Вывод:** отсутствие детей снижает риск проблем с возвратом кредита в срок.

- **7.5%** бездетных кредиторов имели проблемы с возвращением кредита в срок.
- **9.2%** с одним ребёнком
- **9.5%** с двумя детьми
- **8.2%** с тремя детьми (3 и более детей в датасете это маленькая выборка, поэтому результаты менее репрезентативны)
- **9.8%** с четырьмя детьми
- **0%**   с пятью детьми (всего 9 случаев, из них ни одного проблемного)

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### 3.2.1 Мужчины

In [21]:
df_m = df.loc[df['gender'] == 'M']
correlation_pivot(df_m, 'family_status')

,общее число заемщиков,доля должников в %
family_status,,
Не женат / не замужем,1073,14.445480
в разводе,258,8.914729
вдовец / вдова,55,20.000000
гражданский брак,1301,11.760184
женат / замужем,4551,8.811250


##### 3.2.2 Женщины

In [22]:
df_f = df.loc[df['gender'] == 'F']
correlation_pivot(df_f, 'family_status')

,общее число заемщиков,доля должников в %
family_status,,
Не женат / не замужем,1723,6.848520
в разводе,931,6.552095
вдовец / вдова,896,5.803571
гражданский брак,2832,8.192090
женат / замужем,7710,6.822309


**Вывод:** больше всего проблем с выплатой кредита в срок имеют мужчины, особенно из категорий **"вдовец" - 20%** (хотя тут сложно говорить о достоверности ввиду слишком маленькой выборки)  и **"не женат" (14.4%)**, после них идут категории **"гражданский брак" (11.8%)**, **"в разводе" (8.9%)** и **"женат" (8.8%)**.
Женщины более благонадёжны, даже в самой проблемной категории среди женщин **"гражданский брак" (8.2%)**, ситуация всё равно лучше чем самые надёжные категории среди мужчин. Женщина и **женатая** и **"не замужем"** имеет показатель **6.8%**, **в разводе 6.6%**, **вдова 5.8%**.

Похоже что на женщинах семейное положение сказывается в меньшей степени

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
correlation_pivot(df, 'total_income_category')

,общее число заемщиков,доля должников в %
total_income_category,,
A,25,8.000000
B,5014,7.060231
C,15921,8.498210
D,349,6.017192
E,22,9.090909


**Вывод:** распределение по доходам выглядит следующим образом:
- **E** (менее 30 000) --- **9%**
- **D** (30 000 - 50 000) --- **6%**
- **C** (50 000 - 200 000) --- **8.5%**
- **B** (200 000 - 1 000 000) --- **7%**
- **A** (более 1 000 000) --- **8%**

Ввиду слишком маленькой выборки по категориям **E**, **D** и **A**, делать выводы о них не целесообразно. Исходя из данных категорий **C** и **B**, люди с более высоким доходом более надёжны

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [24]:
correlation_pivot(df, 'purpose_category')

,общее число заемщиков,доля должников в %
purpose_category,,
операции с автомобилем,4279,9.347978
операции с недвижимостью,10751,7.255139
получение образования,3988,9.252758
проведение свадьбы,2313,7.911803


**Вывод:** Кредиты на **авто** и **образование** имеют больший процент проблем

## общий вывод

Исследование показало много зависимостей от категории лиц, которые коррелируют с возвратом кредита в срок. 

Самые надёжные категории, с которой менее всего проблем это **люди без детей, женщины, с более высокими доходами, берут кредит на недвижимость или свадьбу**. Таким категориям граждан можно доверять больше.

наименее надёжные категории: **мужчины (особенно не женатые), с детьми, с доходом ниже среднего, берут кредит на авто или учёбу**. Таким категориям граждан стоит доверять меньше.
